# Testing pymysql

## Import the pymysql module

In [2]:
!pip install pymysql --quiet

In [85]:
import pymysql
import pandas as pd
import numpy as np

## Create connection a to the MySQL database

In [185]:
connection = pymysql.connect(host='34.159.115.175',
                             user='freddo',
                             password='batch-735-berlin',
                             db='db-stock-news-pred')

## Create a cursor using the cursor() function

In [186]:
cursor = connection.cursor()

## Execute the required SQL query

In [141]:
# Create a new query that selects the entire contents of 'ticker'
sql = "SELECT * FROM `ticker`"
cursor.execute(sql)

# Fetch all the records and use a for loop to print them one line at a time
result = cursor.fetchall()
for i in result:
    print(i)

('AAPL',)
('AMZN',)
('GOOGL',)
('MSFT',)


# Read the CSV file in a DataFrame

In [90]:
!pwd

/home/freddo/code/freddo1503/stock_news_pred/notebooks


In [174]:
df = pd.read_csv('../../stock_prices.csv', delimiter='\t')

In [178]:
df = df.rename(columns={'Unnamed: 0':'Date'})

In [179]:
df

,Date,AAPL,MSFT,GOOGL,AMZN
0,2021-11-22,161.02,339.83,2926.04,3572.57
1,2021-11-19,160.55,343.11,2978.53,3676.57
2,2021-11-18,157.87,341.27,2996.77,3696.06
3,2021-11-17,153.49,339.12,2960.94,3549.00
4,2021-11-16,151.00,339.51,2957.52,3540.70
...,...,...,...,...,...
5547,1999-11-05,88.31,91.56,NaN,64.94
5548,1999-11-04,83.62,91.75,NaN,63.06
5549,1999-11-03,81.50,92.00,NaN,65.81
5550,1999-11-02,80.25,92.56,NaN,66.44


In [180]:
date = list(df['Date'].values)

In [109]:
type(date[0])

str

In [187]:
date

['2021-11-22',
 '2021-11-19',
 '2021-11-18',
 '2021-11-17',
 '2021-11-16',
 '2021-11-15',
 '2021-11-12',
 '2021-11-11',
 '2021-11-10',
 '2021-11-09',
 '2021-11-08',
 '2021-11-05',
 '2021-11-04',
 '2021-11-03',
 '2021-11-02',
 '2021-11-01',
 '2021-10-29',
 '2021-10-28',
 '2021-10-27',
 '2021-10-26',
 '2021-10-25',
 '2021-10-22',
 '2021-10-21',
 '2021-10-20',
 '2021-10-19',
 '2021-10-18',
 '2021-10-15',
 '2021-10-14',
 '2021-10-13',
 '2021-10-12',
 '2021-10-11',
 '2021-10-08',
 '2021-10-07',
 '2021-10-06',
 '2021-10-05',
 '2021-10-04',
 '2021-10-01',
 '2021-09-30',
 '2021-09-29',
 '2021-09-28',
 '2021-09-27',
 '2021-09-24',
 '2021-09-23',
 '2021-09-22',
 '2021-09-21',
 '2021-09-20',
 '2021-09-17',
 '2021-09-16',
 '2021-09-15',
 '2021-09-14',
 '2021-09-13',
 '2021-09-10',
 '2021-09-09',
 '2021-09-08',
 '2021-09-07',
 '2021-09-03',
 '2021-09-02',
 '2021-09-01',
 '2021-08-31',
 '2021-08-30',
 '2021-08-27',
 '2021-08-26',
 '2021-08-25',
 '2021-08-24',
 '2021-08-23',
 '2021-08-20',
 '2021-08-

# Creating the timeline

Adding the date into the timeline in the date columns

In [165]:
# Insert the list of the date one by one.
for i in date:
    sql = f"INSERT INTO timeline(date) VALUES ('{i}')"
    cursor.execute(sql)

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

# Adding the data into the stockapi

In [189]:
df.head(5)

,Date,AAPL,MSFT,GOOGL,AMZN
0,2021-11-22,161.02,339.83,2926.04,3572.57
1,2021-11-19,160.55,343.11,2978.53,3676.57
2,2021-11-18,157.87,341.27,2996.77,3696.06
3,2021-11-17,153.49,339.12,2960.94,3549.00
4,2021-11-16,151.00,339.51,2957.52,3540.70


In [213]:
aapl_price = list(df[['Date','AAPL']].values)
aapl_price = np.array(aapl_price)
aapl_price = aapl_price.tolist()

In [231]:
msft_price = list(df[['Date','MSFT']].values)
msft_price = np.array(msft_price)
msft_price = msft_price.tolist()

In [253]:
googl_price = df[['Date','GOOGL']]
googl_price = googl_price['GOOGL'].fillna(0)
googl_price = list(df[['Date','GOOGL']].values)
googl_price = np.array(googl_price)
googl_price = googl_price.tolist()

In [216]:
amzn_price = list(df[['Date','AMZN']].values)
amzn_price = np.array(amzn_price)
amzn_price = amzn_price.tolist()

In [218]:
aapl_price[:5]

[['2021-11-22', 161.02],
 ['2021-11-19', 160.55],
 ['2021-11-18', 157.87],
 ['2021-11-17', 153.49],
 ['2021-11-16', 151.0]]

In [234]:
%%time
for i,j in aapl_price:
    sql = f"UPDATE stocksapi \
            SET stock_price={j} \
            WHERE ticker='AAPL' AND `date`='{i}'"
    cursor.execute(sql)

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

CPU times: user 2.34 s, sys: 640 ms, total: 2.98 s
Wall time: 4min 7s


In [235]:
%%time
for i,j in msft_price:
    sql = f"UPDATE stocksapi \
            SET stock_price={j} \
            WHERE ticker='MSFT' AND `date`='{i}'"
    cursor.execute(sql)

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

CPU times: user 2.28 s, sys: 570 ms, total: 2.85 s
Wall time: 4min 9s


In [254]:
%%time
for i,j in googl_price:
    sql = f"UPDATE stocksapi \
            SET stock_price={j} \
            WHERE ticker='GOOGL' AND `date`='{i}'"
    
    cursor.execute(sql)

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

OperationalError: (1054, "Unknown column 'nan' in 'field list'")

In [237]:
%%time
for i,j in amzn_price:
    sql = f"UPDATE stocksapi \
            SET stock_price={j} \
            WHERE ticker='AMZN' AND `date`='{i}'"
    cursor.execute(sql)

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

CPU times: user 2.26 s, sys: 308 ms, total: 2.57 s
Wall time: 4min 39s


In [202]:
# Insert the list of the date one by one.
for i in aapl_price.values:
    sql = f"INSERT INTO timeline(date) VALUES ('{i}')"
    print(i)

['2021-11-22' 161.02]
['2021-11-19' 160.55]
['2021-11-18' 157.87]
['2021-11-17' 153.49]
['2021-11-16' 151.0]
['2021-11-15' 150.0]
['2021-11-12' 149.99]
['2021-11-11' 147.87]
['2021-11-10' 147.92]
['2021-11-09' 150.81]
['2021-11-08' 150.44]
['2021-11-05' 151.28]
['2021-11-04' 150.96]
['2021-11-03' 151.49]
['2021-11-02' 150.02]
['2021-11-01' 148.96]
['2021-10-29' 149.8]
['2021-10-28' 152.57]
['2021-10-27' 148.85]
['2021-10-26' 149.32]
['2021-10-25' 148.64]
['2021-10-22' 148.69]
['2021-10-21' 149.48]
['2021-10-20' 149.26]
['2021-10-19' 148.76]
['2021-10-18' 146.55]
['2021-10-15' 144.84]
['2021-10-14' 143.76]
['2021-10-13' 140.91]
['2021-10-12' 141.51]
['2021-10-11' 142.81]
['2021-10-08' 142.9]
['2021-10-07' 143.29]
['2021-10-06' 142.0]
['2021-10-05' 141.11]
['2021-10-04' 139.14]
['2021-10-01' 142.65]
['2021-09-30' 141.5]
['2021-09-29' 142.83]
['2021-09-28' 141.91]
['2021-09-27' 145.37]
['2021-09-24' 146.92]
['2021-09-23' 146.83]
['2021-09-22' 145.85]
['2021-09-21' 143.43]
['2021-09-20' 14

['2017-03-09' 138.68]
['2017-03-08' 139.0]
['2017-03-07' 139.52]
['2017-03-06' 139.34]
['2017-03-03' 139.78]
['2017-03-02' 138.96]
['2017-03-01' 139.79]
['2017-02-28' 136.99]
['2017-02-27' 136.93]
['2017-02-24' 136.66]
['2017-02-23' 136.53]
['2017-02-22' 137.11]
['2017-02-21' 136.7]
['2017-02-17' 135.72]
['2017-02-16' 135.345]
['2017-02-15' 135.51]
['2017-02-14' 135.02]
['2017-02-13' 133.29]
['2017-02-10' 132.12]
['2017-02-09' 132.42]
['2017-02-08' 132.04]
['2017-02-07' 131.53]
['2017-02-06' 130.29]
['2017-02-03' 129.08]
['2017-02-02' 128.53]
['2017-02-01' 128.75]
['2017-01-31' 121.35]
['2017-01-30' 121.63]
['2017-01-27' 121.95]
['2017-01-26' 121.94]
['2017-01-25' 121.88]
['2017-01-24' 119.97]
['2017-01-23' 120.08]
['2017-01-20' 120.0]
['2017-01-19' 119.78]
['2017-01-18' 119.99]
['2017-01-17' 120.0]
['2017-01-13' 119.04]
['2017-01-12' 119.25]
['2017-01-11' 119.75]
['2017-01-10' 119.11]
['2017-01-09' 118.99]
['2017-01-06' 117.91]
['2017-01-05' 116.61]
['2017-01-04' 116.02]
['2017-01-03'

['2011-10-26' 400.6]
['2011-10-25' 397.77]
['2011-10-24' 405.77]
['2011-10-21' 392.87]
['2011-10-20' 395.31]
['2011-10-19' 398.62]
['2011-10-18' 422.24]
['2011-10-17' 419.99]
['2011-10-14' 422.0]
['2011-10-13' 408.43]
['2011-10-12' 402.19]
['2011-10-11' 400.29]
['2011-10-10' 388.81]
['2011-10-07' 369.8]
['2011-10-06' 377.37]
['2011-10-05' 378.25]
['2011-10-04' 372.5]
['2011-10-03' 374.6]
['2011-09-30' 381.32]
['2011-09-29' 390.57]
['2011-09-28' 397.01]
['2011-09-27' 399.26]
['2011-09-26' 403.17]
['2011-09-23' 404.3]
['2011-09-22' 401.82]
['2011-09-21' 412.14]
['2011-09-20' 413.45]
['2011-09-19' 411.63]
['2011-09-16' 400.5]
['2011-09-15' 392.96]
['2011-09-14' 389.3]
['2011-09-13' 384.62]
['2011-09-12' 379.94]
['2011-09-09' 377.48]
['2011-09-08' 384.14]
['2011-09-07' 383.93]
['2011-09-06' 379.74]
['2011-09-02' 374.05]
['2011-09-01' 381.03]
['2011-08-31' 384.83]
['2011-08-30' 389.99]
['2011-08-29' 389.97]
['2011-08-26' 383.58]
['2011-08-25' 373.72]
['2011-08-24' 376.18]
['2011-08-23' 373.

['2005-10-14' 54.0]
['2005-10-13' 53.74]
['2005-10-12' 49.25]
['2005-10-11' 51.59]
['2005-10-10' 50.37]
['2005-10-07' 51.3]
['2005-10-06' 51.7]
['2005-10-05' 52.7801]
['2005-10-04' 53.75]
['2005-10-03' 54.44]
['2005-09-30' 53.61]
['2005-09-29' 52.34]
['2005-09-28' 51.08]
['2005-09-27' 53.44]
['2005-09-26' 53.84]
['2005-09-23' 53.2]
['2005-09-22' 51.9]
['2005-09-21' 52.11]
['2005-09-20' 53.19]
['2005-09-19' 52.64]
['2005-09-16' 51.21]
['2005-09-15' 49.87]
['2005-09-14' 49.61]
['2005-09-13' 50.82]
['2005-09-12' 51.4]
['2005-09-09' 51.31]
['2005-09-08' 49.78]
['2005-09-07' 48.68]
['2005-09-06' 48.8]
['2005-09-02' 46.22]
['2005-09-01' 46.26]
['2005-08-31' 46.89]
['2005-08-30' 46.57]
['2005-08-29' 45.84]
['2005-08-26' 45.74]
['2005-08-25' 46.06]
['2005-08-24' 45.77]
['2005-08-23' 45.74]
['2005-08-22' 45.87]
['2005-08-19' 45.83]
['2005-08-18' 46.3]
['2005-08-17' 47.15]
['2005-08-16' 46.25]
['2005-08-15' 47.68]
['2005-08-12' 46.1]
['2005-08-11' 44.0]
['2005-08-10' 43.38]
['2005-08-09' 43.82]


['2001-06-04' 20.66]
['2001-06-01' 20.89]
['2001-05-31' 19.95]
['2001-05-30' 19.78]
['2001-05-29' 21.47]
['2001-05-25' 22.76]
['2001-05-24' 23.2]
['2001-05-23' 23.23]
['2001-05-22' 23.5]
['2001-05-21' 23.56]
['2001-05-18' 23.53]
['2001-05-17' 23.55]
['2001-05-16' 24.1]
['2001-05-15' 23.18]
['2001-05-14' 23.29]
['2001-05-11' 22.85]
['2001-05-10' 23.0]
['2001-05-09' 23.98]
['2001-05-08' 24.57]
['2001-05-07' 24.96]
['2001-05-04' 25.75]
['2001-05-03' 24.96]
['2001-05-02' 26.59]
['2001-05-01' 25.93]
['2001-04-30' 25.49]
['2001-04-27' 26.2]
['2001-04-26' 24.69]
['2001-04-25' 24.72]
['2001-04-24' 24.03]
['2001-04-23' 24.25]
['2001-04-20' 25.04]
['2001-04-19' 25.72]
['2001-04-18' 22.79]
['2001-04-17' 20.4]
['2001-04-16' 21.44]
['2001-04-12' 22.42]
['2001-04-11' 21.8]
['2001-04-10' 22.04]
['2001-04-09' 20.54]
['2001-04-06' 20.59]
['2001-04-05' 20.87]
['2001-04-04' 19.5]
['2001-04-03' 20.24]
['2001-04-02' 21.59]
['2001-03-30' 22.07]
['2001-03-29' 22.53]
['2001-03-28' 22.17]
['2001-03-27' 22.87]


In [ ]:
    cursor.execute(sql)

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

# Don't forget to close the connection

In [ ]:
# Close the connection
connection.close()